In [107]:
import pandas as pd
import xlwings as xw
import os
import shutil
import random

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

df = pd.read_excel('2014.01.21 - Gara Carbonera/ISCRITTI 1 TROFEO CARBONERA 2024.xlsx')
df['CINTURA'] = df['CINTURA'].replace(to_replace='VERDE VE-BL', value='VERDE VE-BLU', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='MARRONE NERA', value='MARRONE e NERA', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='MARRONE E NERA', value='MARRONE e NERA', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='GIALLA GI-ARANCIO', value='GIALLA GI-AR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BLU BL-MA', value='BLU BLU-MAR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BIANCA e BI-GI', value='BIANCA BI-GI', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='ARANCIO A-V', value='ARANCIO AR-VE', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='KARATE SACILE FONTANAFREDDA\xa0', value='KARATE SACILE FONTANAFREDDA', regex=False)
df.rename(columns={"SOCIETA'": "SOCIETA"}, inplace=True)
df.loc[(df['CINTURA'] == 'MARRONE e NERA') & (df['M / F'] == 'M'), 'CINTURA'] = 'MARRONE e NERA MASCHILE'
df.loc[(df['CINTURA'] == 'MARRONE e NERA') & (df['M / F'] == 'F'), 'CINTURA'] = 'MARRONE e NERA FEMMINILE'
df.drop(columns=['ISCRITTO'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.index.name = 'ID'
df.head()

c:\Users\932197\Documents\dati_locali\git-hub\karate\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,COGNOME,NOME,ANNO,M / F,CINTURA,SOCIETA,QUOTA DI ISCRIZIONE,FASCIA
ID,,,,,,,,
0,PIERAGNOLO,LEONE,2015,M,VERDE VE-BLU,SCUOLA KARATE MIGNAGOLA,NaN,FASCIA C
1,RONCATO,ELISA,2014,F,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
2,STEINER,FEDERICO,2015,M,ARANCIO AR-VE,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
3,BUCCI,GIOIA,2013,M,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA D
4,BUCCI,FRANCESCO,2016,M,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA B


In [108]:
df_temp = pd.read_excel("2014.01.21 - Gara Carbonera/Tabellone vuoto.xlsx", sheet_name='Foglio3')
fasce = df_temp['Fascia'].dropna()
anni = df_temp['Anno'].dropna()
categorie = df_temp['Categoria'].dropna()

In [109]:
def ricrea_cartella_tabelloni():
    if os.path.exists('2014.01.21 - Gara Carbonera/Tabelloni generati'):
        shutil.rmtree('2014.01.21 - Gara Carbonera/Tabelloni generati')
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati')   
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati/PDF')   
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati/Excel')   
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati/Parquet')  

def iscritti_per_tabellone(fascia, categoria):
    iscritti = df[(df['FASCIA'] == f'FASCIA {fascia}') & (df['CINTURA'] == categoria)].copy()
    n_iscritti = len(iscritti)
    atleta_singolo = None
    if n_iscritti % 2 == 1:
        # Se sono DISPARI, sceglie l'atleta che non farà il primo incontro fra quelli della società col maggior numero di atleti iscritti
        soc_maggiori_iscritti = iscritti.groupby(['SOCIETA'])[['SOCIETA']].value_counts().idxmax()
        id_atleta_singolo = iscritti[iscritti['SOCIETA'] == soc_maggiori_iscritti].index[0]
        atleta_singolo = iscritti.loc[id_atleta_singolo]
        iscritti.drop(id_atleta_singolo, inplace=True)

    iscritti['ISCRITTI_PER_SOCIETA'] = iscritti.groupby('SOCIETA')['SOCIETA'].transform('count')
    iscritti = iscritti.sort_values(by='ISCRITTI_PER_SOCIETA', ascending=False)
    
    return iscritti, atleta_singolo

def salva_tabellone(app, tab, n_tab, fascia, categoria):
    tab_vuoto = '2014.01.21 - Gara Carbonera/Tabellone vuoto.xlsx'
    foglio = 'TAB.SINGOLI'
    nome_file = f'{str(n_tab).zfill(2)}) FASCIA {fascia} - {categoria}'
    riga_inizio_tabella = 8
    riga_fine_tabella = 117
    
    wb = app.books.open(tab_vuoto)
    sheet = wb.sheets[foglio]  
    sheet['C1'].value = fascia
    sheet['F1'].value = categoria
    sheet[f'A{riga_inizio_tabella}'].value = tab.values
    sheet['J1'].value = n_tab
    prima_riga_vuota = riga_inizio_tabella
    for riga in range(riga_inizio_tabella, riga_fine_tabella):
        if (not sheet[f'C{riga}'].value) and (sheet[f'B{riga}'].value == 'R'):
            prima_riga_vuota = riga
            break
    rg = sheet.range(f'{prima_riga_vuota}:{riga_fine_tabella}')
    rg.delete(shift='up')
    sheet.to_pdf(f'2014.01.21 - Gara Carbonera/Tabelloni generati/PDF/{nome_file}.pdf')
    wb.save(f'2014.01.21 - Gara Carbonera/Tabelloni generati/Excel/{nome_file}.xlsx')
    tab.to_parquet(f'2014.01.21 - Gara Carbonera/Tabelloni generati/Parquet/{nome_file}.parquet')
    wb.close()


In [110]:
ricrea_cartella_tabelloni()
tot_tabelloni = len(fasce) * len(categorie)
n_tab = 1
n_tab_salvati = 1
app = xw.App(visible=False)
for fascia in fasce:
    for categoria in categorie:
        print(f'Tabellone n.{n_tab} di {tot_tabelloni}: Fascia {fascia}, Categoria: {categoria}')
        iscritti, atleta_singolo = iscritti_per_tabellone(fascia, categoria)
        tot_incontri = len(iscritti) // 2
        accoppiati = []
        riga = {}
        tab = pd.DataFrame(columns=['N_INCONTRO', 'CINTURA', 'SOCIETA', 'SOC2', 'SOC3', 'COGNOME', 'NOME'])
        tab_lista = []
        n_incontro = 0
        if (not iscritti.empty) or (atleta_singolo is not None):
            for n_incontro in range(1, tot_incontri+1):
                for i in range(len(iscritti)):
                    atleta1 = iscritti.iloc[i]
                    if atleta1.name not in accoppiati:
                        accoppiati.append(atleta1.name)
                        break
                df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] != atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
                trovato = False
                for i in range(len(df_atleta2)):
                    atleta2 = df_atleta2.iloc[i]
                    if atleta2.name not in accoppiati:
                        accoppiati.append(atleta2.name)
                        trovato = True
                        break        
                if not trovato:
                    df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] == atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
                    for i in range(len(df_atleta2)):
                        atleta2 = df_atleta2.iloc[i]
                        if atleta2.name not in accoppiati:
                            accoppiati.append(atleta2.name)
                            break        
                tab.loc[len(tab)] = [n_incontro, 'R', atleta1['SOCIETA'].upper(), '', '', atleta1['COGNOME'].upper(), atleta1['NOME'].upper()]
                tab.loc[len(tab)] = [n_incontro, 'B', atleta2['SOCIETA'].upper(), '', '', atleta2['COGNOME'].upper(), atleta2['NOME'].upper()]
            
            if not iscritti.empty: # Se c'è un unico iscritto non serve mescolare
                # Mescola l'ordine degli incontri
                pairs = [tab.iloc[i:i+2] for i in range(0, len(tab), 2)]
                random.shuffle(pairs)
                random.shuffle(pairs)
                random.shuffle(pairs)
                random.shuffle(pairs)
                random.shuffle(pairs)
                tab = pd.concat(pairs).reset_index(drop=True)
                n = 1
                for i in range(0, len(tab), 2):
                    tab.loc[i, 'N_INCONTRO'] = n
                    tab.loc[i+1, 'N_INCONTRO'] = n
                    n += 1

            # Se il numero di atleti è dispari aggiunge alla fine l'atleta della società col numero maggiore di partecipanti
            if atleta_singolo is not None:
                n_incontro += 1
                tab.loc[len(tab)] = [n_incontro, 'R', atleta_singolo['SOCIETA'].upper(), '', '', atleta_singolo['COGNOME'].upper(), atleta_singolo['NOME'].upper()]
                
            salva_tabellone(app, tab, n_tab_salvati, fascia, categoria)
            n_tab_salvati += 1
        n_tab += 1
app.quit()

Tabellone n.1 di 35: Fascia A, Categoria: BIANCA BI-GI
Tabellone n.2 di 35: Fascia A, Categoria: GIALLA GI-AR
Tabellone n.3 di 35: Fascia A, Categoria: ARANCIO AR-VE
Tabellone n.4 di 35: Fascia A, Categoria: VERDE VE-BLU
Tabellone n.5 di 35: Fascia A, Categoria: BLU BLU-MAR
Tabellone n.6 di 35: Fascia A, Categoria: MARRONE e NERA MASCHILE
Tabellone n.7 di 35: Fascia A, Categoria: MARRONE e NERA FEMMINILE
Tabellone n.8 di 35: Fascia B, Categoria: BIANCA BI-GI
Tabellone n.9 di 35: Fascia B, Categoria: GIALLA GI-AR
Tabellone n.10 di 35: Fascia B, Categoria: ARANCIO AR-VE
Tabellone n.11 di 35: Fascia B, Categoria: VERDE VE-BLU
Tabellone n.12 di 35: Fascia B, Categoria: BLU BLU-MAR
Tabellone n.13 di 35: Fascia B, Categoria: MARRONE e NERA MASCHILE
Tabellone n.14 di 35: Fascia B, Categoria: MARRONE e NERA FEMMINILE
Tabellone n.15 di 35: Fascia C, Categoria: BIANCA BI-GI
Tabellone n.16 di 35: Fascia C, Categoria: GIALLA GI-AR
Tabellone n.17 di 35: Fascia C, Categoria: ARANCIO AR-VE
Tabellone 

iscritti, atleta_singolo = iscritti_per_tabellone('C', 'GIALLA GI-AR')

iscritti


tot_incontri = len(iscritti) // 2
accoppiati = []
riga = {}
tab = pd.DataFrame(columns=['N_INCONTRO', 'CINTURA', 'SOCIETA', 'SOC2', 'SOC3', 'COGNOME', 'NOME'])
tab_lista = []
if (not iscritti.empty) or (atleta_singolo is not None):
    for n_incontro in range(1, tot_incontri+1):
        for i in range(len(iscritti)):
            atleta1 = iscritti.iloc[i]
            if atleta1.name not in accoppiati:
                accoppiati.append(atleta1.name)
                break
        df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] != atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
        trovato = False
        for i in range(len(df_atleta2)):
            atleta2 = df_atleta2.iloc[i]
            if atleta2.name not in accoppiati:
                accoppiati.append(atleta2.name)
                trovato = True
                break        
        if not trovato:
            df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] == atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
            for i in range(len(df_atleta2)):
                atleta2 = df_atleta2.iloc[i]
                if atleta2.name not in accoppiati:
                    accoppiati.append(atleta2.name)
                    break        
        tab.loc[len(tab)] = [n_incontro, 'R', atleta1['SOCIETA'].upper(), '', '', atleta1['COGNOME'].upper(), atleta1['NOME'].upper()]
        tab.loc[len(tab)] = [n_incontro, 'B', atleta2['SOCIETA'].upper(), '', '', atleta2['COGNOME'].upper(), atleta2['NOME'].upper()]
    
    if not iscritti.empty: # Se c'è un unico iscritto non serve mescolare
        # Mescola l'ordine degli incontri
        pairs = [tab.iloc[i:i+2] for i in range(0, len(tab), 2)]
        random.shuffle(pairs)
        shuffled_df = pd.concat(pairs).reset_index(drop=True)
        n = 1
        for i in range(0, len(shuffled_df), 2):
            shuffled_df.loc[i, 'N_INCONTRO'] = n
            shuffled_df.loc[i+1, 'N_INCONTRO'] = n
            n += 1


tab

shuffled_df